# 考察

極端に上がりの速さが遅いクラスターの分析を行う．

## 環境構築

In [1]:
# Notebook初期設定
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings('ignore')

In [60]:
import os
import pandas as pd
import sqlite3
from tqdm import tqdm_notebook as tqdm

In [3]:
# plotly関連
!pip install plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [4]:
DIR_IN = '../data/scatter_preprocess'
DIR_OUT = '../data/scatter_discussion'

In [54]:
FN_ALL = 'all_res.csv'

FN_DB = 'keiba.db'
TN_RACE = 'race'
TN_RES = 'result'

In [61]:
def make_df_by_query(path_db, query):
    """path_dbにqueryを投げた結果のdfを取得"""
    conn = sqlite3.connect(path_db, isolation_level=None)
    cur = conn.cursor()
    df = pd.read_sql(query, conn)
    return df

## 分析

In [62]:
df_all = pd.read_csv(os.path.join(DIR_IN, FN_ALL))

In [63]:
# speed_3fのレースごとの最大値を抜き出す
df_max_speed_3f = df_all.groupby('race_id')['speed_3f'].max().reset_index()
# speed_3fの最大値が50km/h以下のrace_idを抽出
raceids_slow_3f = \
    set(df_max_speed_3f[df_max_speed_3f['speed_3f'] < 50]['race_id'].unique())

In [64]:
# raceテーブルの全情報を取得
path_db = os.path.join(DIR_IN, FN_DB)
df_race = make_df_by_query(
    path_db, 'SELECT * FROM race;')

In [69]:
# raceids_slow_3fに属するraceのみ抽出
df_race_slow_3f = \
    df_race[df_race['race_id'].astype(int).isin(raceids_slow_3f)].\
    reset_index(drop=True)

In [73]:
df_race_slow_3f.shape

(351, 17)

In [78]:
# 開催年と開催場所で集計してみる
df_race_slow_3f['year'] = \
    pd.to_datetime(df_race_slow_3f['date']).dt.year
df_race_slow_3f.groupby(['year', 'place'])['race_id'].count()

year  place
1986  京都       16
      新潟        1
      東京       19
      阪神       15
1987  京都       16
      東京       21
      阪神       17
1988  京都       17
      新潟        6
      東京       30
      阪神       17
1989  京都       17
      新潟        4
      東京       21
      阪神       17
1990  京都       18
      新潟        2
      東京        6
      阪神       12
1991  京都       22
      新潟        3
      阪神        4
1992  京都       17
      新潟        3
      阪神       17
1993  京都        6
      阪神        7
Name: race_id, dtype: int64

In [79]:
# 出力
path_out = os.path.join(DIR_OUT, 'race_slow_3f.csv')
df_race_slow_3f[['race_id', 'race_name', 'date', 'title', 'place']].\
    to_csv(path_out, index=False)